In [1]:
import pandas as pd
import numpy as np

   首先我们最直接的就是查看我们右上角帮助里的pythonAPI文档， 里面的阐述的比较具体。
今天我们先用notebook实现这几点：
## 1 直接通过财务指标选取出来符合要求的股票（从单一到复合）
## 2 在已有的股票池中选取符合要求的股票
#### 我们先了解一下get_fundamentals()里的几个参数
   * query 是你要去查找的财务数据
   * entry_date 开始查找的日期，默认是NONE，在Notebook里是必须要填写的。在策略里是你回测的起始日期
   * interval 查询频率，比如说5天一查，5年一查
   * report_quarter 返回的时候是否返回是哪一个季度的


In [2]:
df = get_fundamentals(
    query(
        #在这里选取你要你要的财务指标，详情参考财务数据文档，我们先在这里按照市值大小来选取，market_cap
        fundamentals.eod_derivative_indicator.market_cap
    ),entry_date='2015-06-01',interval='5y'
)

In [3]:
df.minor_axis

Index(['002481.XSHE', '000955.XSHE', '600215.XSHG', '603818.XSHG',
       '000540.XSHE', '000953.XSHE', '300168.XSHE', '600097.XSHG',
       '000046.XSHE', '002697.XSHE',
       ...
       '603123.XSHG', '600403.XSHG', '600525.XSHG', '300374.XSHE',
       '002202.XSHE', '002726.XSHE', '000958.XSHE', '002193.XSHE',
       '002187.XSHE', '600696.XSHG'],
      dtype='object', length=2748)

In [4]:
df.major_axis

DatetimeIndex(['2015-06-01', '2014-05-30', '2013-05-30', '2012-05-30',
               '2011-05-30'],
              dtype='datetime64[ns]', freq=None)

#### 注意这里是返回的是一个Panel,因为我们查询的是一个时间，股票代码，还有市值的一个三维的集，所以一定是会返回一个pannel那我们如果要返回一个dataFrame，那么我们就要减少一个维度，比如说，我们要的是300281.XSHE这一只股票

In [5]:
df.minor_xs('300281.XSHE')

,market_cap
2015-06-01,4.80118e+09
2014-05-30,1.5136e+09
2013-05-30,1.314e+09
2012-05-30,1.0758e+09
2011-05-30,NaN


In [6]:
stocks=np.array(df.minor_axis)
stocks

array(['002481.XSHE', '000955.XSHE', '600215.XSHG', ..., '002193.XSHE',
       '002187.XSHE', '600696.XSHG'], dtype=object)

#### 同理，我们要获取某一个时间的话就是用df.major_xs('2014-05-30')
这里，我们终于获得了API里面写的dataFrame了....即使很心塞，但是不得不承认完善的情况下是一定是三维的。
那么接下来我们通过财务数据最终是要选取股票出来，那么其实我们就是要获取到查询出来的这个panel里的minor_axis

这里我们使用numpy里的array来保存一下这个股票列表。（因为array保存的是相同的数据类型，list可以不是）
顺带一提我们经常在社区里看到的策略都有context.stocks = 一个DataFrame.columns.values，因为这是二维dataFrame获取的是列的值就是股票列表了

#### 下面我们直接来一个API里复合查询的demo：
   * filter方法是可以让你去筛选指标的，比如说demo里就是筛选pe大于55小于60之间
   * .order_by方法是一个排序的方法，demo里是用revenue来进行一个降序排列 desc()是降序，asc()是升序。
   * .limit方式是限制拿到数据的数量，因为最终你可能只需要排名靠前的10只

In [7]:
fundamental_df = get_fundamentals(
        query(
            fundamentals.income_statement.revenue, fundamentals.eod_derivative_indicator.pe_ratio
        ).filter(
            fundamentals.eod_derivative_indicator.pe_ratio > 55
        ).filter(
            fundamentals.eod_derivative_indicator.pe_ratio < 60
        ).order_by(
            fundamentals.income_statement.revenue.desc()
        ).limit(
            10
        ),entry_date='2016-04-01',interval='5y'  #策略里填写如果只需要股票池，那么entry_date和interval是不需要填写的，但是notebook中是要的
    )


In [8]:
df=get_fundamentals(query(fundamentals.eod_derivative_indicator.announce_date,fundamentals.eod_derivative_indicator.pe_ratio).\
                    filter(fundamentals.eod_derivative_indicator.pb_ratio<50).order_by(fundamentals.eod_derivative_indicator.pb_ratio).limit(10),\
                   entry_date='2014-04-01',interval='1y')


In [9]:
df

<class 'pandas.core.panel.Panel'>
Dimensions: 2 (items) x 1 (major_axis) x 10 (minor_axis)
Items axis: ANNOUNCE_DATE to pe_ratio
Major_axis axis: 2014-04-01 00:00:00 to 2014-04-01 00:00:00
Minor_axis axis: 000898.XSHE to 600005.XSHG

In [10]:
df.iloc[0]

,000898.XSHE,000709.XSHE,600308.XSHG,000932.XSHE,600808.XSHG,600219.XSHG,000761.XSHE,000825.XSHE,600269.XSHG,600005.XSHG
2014-04-01,2.01403e+07,2.0131e+07,2.0131e+07,2.01403e+07,2.01403e+07,2.01403e+07,2.0131e+07,2.0131e+07,2.01404e+07,2.0131e+07


In [11]:
df.items,df.major_axis,df.minor_axis

(Index(['ANNOUNCE_DATE', 'pe_ratio'], dtype='object'),
 DatetimeIndex(['2014-04-01'], dtype='datetime64[ns]', freq=None),
 Index(['000898.XSHE', '000709.XSHE', '600308.XSHG', '000932.XSHE',
        '600808.XSHG', '600219.XSHG', '000761.XSHE', '000825.XSHE',
        '600269.XSHG', '600005.XSHG'],
       dtype='object'))

In [12]:
df

<class 'pandas.core.panel.Panel'>
Dimensions: 2 (items) x 1 (major_axis) x 10 (minor_axis)
Items axis: ANNOUNCE_DATE to pe_ratio
Major_axis axis: 2014-04-01 00:00:00 to 2014-04-01 00:00:00
Minor_axis axis: 000898.XSHE to 600005.XSHG

In [13]:
df.iloc[:,0].head()

,ANNOUNCE_DATE,pe_ratio
000898.XSHE,2.01403e+07,27.248
000709.XSHE,2.0131e+07,NaN
600308.XSHG,2.0131e+07,55.338
000932.XSHE,2.01403e+07,51.6005
600808.XSHG,2.01403e+07,79.8378


## fundamentals 方法

#### 这里依然是一个panel，那么我们先把股票池筛选出来,跟上面一样我们用一个array来存


In [14]:
stocks = np.array(fundamental_df.minor_axis)
stocks

array(['601866.XSHG', '600704.XSHG', '601727.XSHG', '600011.XSHG',
       '000626.XSHE', '600998.XSHG', '600297.XSHG', '600875.XSHG',
       '601808.XSHG'], dtype=object)

到这里，我们已经把第一步 从选财务数据中选出符合我们要求的股票完成了。
#### 接下来进行第二步，从我们已有的股票池里选取符合财务数据要求的。
   * 1.我们可以自己维护一个股票，比如在一开始定义context.stocks=['XXXXXX.XSHG',‘XXXX.XSHE’]你想要去查询的股票同样，这里我们直接选取某一个概念里的所有股票去找到里面符合我们财务数据要求的股票
      

In [38]:
stocks

['603421.XSHG',
 '603843.XSHG',
 '603701.XSHG',
 '603016.XSHG',
 '603098.XSHG',
 '603888.XSHG',
 '603033.XSHG',
 '603389.XSHG',
 '601966.XSHG',
 '603101.XSHG',
 '603319.XSHG',
 '603778.XSHG',
 '603861.XSHG',
 '603726.XSHG',
 '603878.XSHG',
 '603159.XSHG',
 '603928.XSHG',
 '603416.XSHG',
 '603959.XSHG',
 '603716.XSHG',
 '603887.XSHG',
 '603258.XSHG',
 '603608.XSHG',
 '603569.XSHG',
 '603028.XSHG',
 '603322.XSHG',
 '601500.XSHG',
 '603859.XSHG',
 '600977.XSHG',
 '603189.XSHG',
 '603007.XSHG',
 '603663.XSHG',
 '603069.XSHG',
 '603868.XSHG',
 '600909.XSHG',
 '601882.XSHG',
 '603738.XSHG',
 '601611.XSHG',
 '603313.XSHG',
 '603160.XSHG',
 '603819.XSHG',
 '601127.XSHG',
 '601997.XSHG',
 '603816.XSHG',
 '603060.XSHG',
 '603660.XSHG',
 '600919.XSHG',
 '603909.XSHG',
 '600908.XSHG',
 '603900.XSHG',
 '603377.XSHG',
 '603777.XSHG',
 '603031.XSHG',
 '603822.XSHG',
 '603090.XSHG',
 '603520.XSHG',
 '603067.XSHG',
 '601900.XSHG',
 '600936.XSHG',
 '603323.XSHG',
 '603667.XSHG',
 '603585.XSHG',
 '603203

In [56]:
all_instruments(type='CS').head()

,abbrev_symbol,board_type,concept_names,de_listed_date,exchange,industry_code,industry_name,listed_date,order_book_id,round_lot,sector_code,sector_code_name,special_type,status,symbol,type
0,HTWY,MainBoard,null,0000-00-00,XSHG,C14,食品制造业,2014-02-11,603288.XSHG,100.0,ConsumerStaples,必需消费品,Normal,Active,海天味业,CS
1,SDHH,MainBoard,黄河三角|基因概念|海水淡化|油气改革,0000-00-00,XSHE,C26,化学原料及化学制品制造业,1998-07-03,000822.XSHE,100.0,Materials,原材料,Normal,Active,山东海化,CS
2,LKXC,GEM,雄安新区|铁路基建,0000-00-00,XSHE,C26,化学原料及化学制品制造业,2015-04-23,300446.XSHE,100.0,Materials,原材料,Normal,Active,乐凯新材,CS
3,BYYS,MainBoard,null,0000-00-00,XSHG,C32,有色金属冶炼和压延加工业,2017-02-15,601212.XSHG,100.0,Materials,原材料,Normal,Active,白银有色,CS
4,SSGF,MainBoard,新三板|低碳经济|锂电池|分拆上市|特斯拉|民营银行|创投概念|稀缺资源|新能源,0000-00-00,XSHG,C38,电气机械及器材制造业,1996-01-30,600884.XSHG,100.0,InformationTechnology,信息技术,Normal,Active,杉杉股份,CS


In [57]:
instruments('000001.XSHE')

Instrument(status='Active', concept_names='基金重仓|外资背景|深圳本地|券商重仓|本月解禁|保险重仓|社保重仓', order_book_id='000001.XSHE', symbol='平安银行', shenwan_industry_code='801780.INDX', special_type='Normal', exchange='XSHE', sector_code_name='金融', de_listed_date='0000-00-00', shenwan_industry_name='银行', abbrev_symbol='PAYH', listed_date='1991-04-03', board_type='MainBoard', sector_code='Financials', industry_code='J66', round_lot=100.0, type='CS', industry_name='货币金融服务')

In [60]:
sector('Financials')[:5]

['000686.XSHE', '600340.XSHG', '600173.XSHG', '000667.XSHE', '600848.XSHG']

In [39]:
stocks=concept('次新股')
stocks[:5]

['603421.XSHG', '603843.XSHG', '603701.XSHG', '603016.XSHG', '603098.XSHG']

 * 2.拿到之后我们就去查询：先解释一下不同的地方
    #### 我们一眼就看到关键的不同在于多了一个.filter()方法，我们是从具体指标的上一级中的方法.stockcode.in(参数)中查询我们传进去的那个股票列表
    #### 同样，这里查询的有两个指标，对应的上级也不一样，那么无论我们用fundamentals.eod_derivative_indicator.stockcode.in还是fundamentals.income_statement.stockcode.in返回的结果都一样

### 呃，这里为了比较是不是真的成功了的话，我们可以再从整个市场里选取一下同样财务指标的股票，看看数量就知道是不是真的从我们自己定义的股票里选取了

In [ ]:
old_fundamental_df = get_fundamentals(
        query(
            fundamentals.income_statement.revenue,      fundamentals.eod_derivative_indicator.pe_ratio
        ).filter(
            fundamentals.eod_derivative_indicator.pe_ratio > 5
        ).filter(
            fundamentals.eod_derivative_indicator.pe_ratio < 300
        ),entry_date='2015-04-01'
    )
stocks2=np.array(old_fundamental_df.minor_axis)
len(stocks2)

# 搞完正经事情之后，我们来一些骚操作，比如说在.filter()加一些计算，或者是拿不同的指标进行比较！

In [ ]:
old_fundamental_df = get_fundamentals(
        query(
            fundamentals.income_statement.revenue,      fundamentals.eod_derivative_indicator.pe_ratio
        ).filter(
            fundamentals.eod_derivative_indicator.pe_ratio +10  > 5
        ).filter(
            fundamentals.eod_derivative_indicator.pe_ratio < 300
        ),entry_date='2015-04-01'
    )
stocks2=np.array(old_fundamental_df.minor_axis)
len(stocks2)

#### 还真可以........其实用处不大，但是就是想试试。

In [ ]:
old_fundamental_df = get_fundamentals(
        query(
            fundamentals.income_statement.revenue,      fundamentals.eod_derivative_indicator.pe_ratio
        ).filter(
            fundamentals.eod_derivative_indicator.pe_ratio < fundamentals.income_statement.revenue
        ),entry_date='2015-04-01',interval='1y'
    )
stocks2=np.array(old_fundamental_df.minor_axis)
len(stocks2)

In [ ]:
import math
old_fundamental_df = get_fundamentals(
        query(
             fundamentals.eod_derivative_indicator.pb_ratio,     
        ).filter(
         fundamentals.eod_derivative_indicator.pb_ratio  >100
        ),entry_date='2015-04-01',interval='1y'
    )
stocks2=np.array(old_fundamental_df.minor_axis)
len(stocks2)

### 实验证明，开个根号是不行的[微笑]，终于试出来了....